<a href="https://colab.research.google.com/github/abdulkadirdemirci/abdulkadirdemirci/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# sentment analizi ve sınıflandırma modelleri

# kütüphaneler

In [18]:
from textblob import TextBlob
from sklearn import model_selection,preprocessing,linear_model,naive_bayes,metrics
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn import decomposition, ensemble

import pandas ,xgboost,numpy,textblob,string
from keras.preprocessing import text,sequence
from keras import layers,models,optimizers

In [4]:
import pandas as pd
data1=pd.read_csv("/content/drive/MyDrive/veriler/metin_madenciligi/train.tsv",sep="\t")
data1.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


sentiment bir etiket ama biz mevcut olanları görmezden gelicez çünkü biz binary sınıflandırma yapıcaz. mevcut veride 01234 seklinde etiketlenmiş 0-1 negatif yorumlar 2 orta 3-4 ise pozitif yorumlar, biz 0-1 neg. 3-4 pozi. alıcaz 2 olanları silicez

In [5]:
data1["Sentiment"].replace(0,"negatif",inplace=True)
data1["Sentiment"].replace(1,"negatif",inplace=True)

data1["Sentiment"].replace(3,"pozitif",inplace=True)
data1["Sentiment"].replace(4,"pozitif",inplace=True)

data1=data1[ (data1.Sentiment=="pozitif") | (data1.Sentiment=="negatif") ]
data1.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,negatif
21,22,1,good for the goose,pozitif
22,23,1,good,pozitif
33,34,1,"the gander , some of which occasionally amuses...",negatif
46,47,1,amuses,pozitif


In [6]:
data1.groupby("Sentiment").count()

,PhraseId,SentenceId,Phrase
Sentiment,,,
negatif,34345,34345,34345
pozitif,42133,42133,42133


In [7]:
df=pd.DataFrame()
df["text"]=data1["Phrase"]
df["labels"]=data1["Sentiment"]
df.head()

,text,labels
0,A series of escapades demonstrating the adage ...,negatif
21,good for the goose,pozitif
22,good,pozitif
33,"the gander , some of which occasionally amuses...",negatif
46,amuses,pozitif


#metin ön işleme

In [8]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
sw=stopwords.words("english")
from textblob import Word
import pandas as pd
!pip install nltk
import nltk
!pip install textblob
import textblob
from textblob import TextBlob
from nltk.stem import PorterStemmer
st=PorterStemmer()
from textblob import Word
nltk.download("wordnet")
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [9]:
# küçük buyuk dönüşümü
df["text"]=df["text"].apply(lambda x : " ".join(x.lower() for x in x.split()))
# noktalama işaretleri
df["text"]=df["text"].str.replace("[^\w\s]","")
# sayılar
df["text"]=df["text"].str.replace("\d","")
# stopwords işlemi
df["text"]=df["text"].apply(lambda x : " ".join(x for x in x.split() if x not in sw))
# seyrekleri silinmesi
silinecek=pd.Series(" ".join(df["text"]).split()).value_counts()[-1000:]
df["text"]=df["text"].apply(lambda x : " ".join(x for x in x.split() if x not in silinecek ))
# kelime kökler
df["text"].apply(lambda x : " ".join(Word(word).lemmatize() for word in x.split()))
df.head(20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


,text,labels
0,series demonstrating adage good goose also goo...,negatif
21,good goose,pozitif
22,good,pozitif
33,gander occasionally amuses none amounts much s...,negatif
46,amuses,pozitif
47,none amounts much story,negatif
49,none amounts much story,negatif
63,quiet introspective entertaining independent w...,pozitif
64,quiet introspective entertaining independent,pozitif
66,quiet introspective entertaining independent,pozitif


# feature engineering


1.   **count vectors**: tüm metinde geçen unique kelimeleri bulup bunları yeni sutunlar olarak veriye ekliyoruz sonra herbir satırdaki cümle yorum vs içinde gecen unique kelimeler aynı adı tasıyon sutunlarda kac kez gectiyse o degeri alıyor.
2.   **TF_IDF VECTORS**(words,charakters,n-grams):


>> TF(t): (bir terimin bir dökümanda gözlenme frekansı) / (dökümandaki toplam terim sayısı)

>> IDF(t): log_e(toplam döküman sayısı / içinde t terimi olan balge sayısı)

3.    Word Embeddings

## train test seti oluşturmak

In [39]:
X_train,X_test,y_train,y_test=model_selection.train_test_split(df["text"],df["labels"],random_state=1)

In [40]:
X_train.head()

118788    presents fascinating glimpse urban life class ...
89514     hey everybody wanna watch movie guy dressed ch...
86857                             incredible number stories
140626                         ultimately disappoint action
153243                                              unified
Name: text, dtype: object

## encoding

In [41]:
encoder= preprocessing.LabelEncoder()

In [42]:
y_train=encoder.fit_transform(y_train)
y_test=encoder.fit_transform(y_test)

In [43]:
y_train[:10]
y_test[:10]

array([1, 0, 1, 0, 0, 0, 0, 1, 0, 1])

## count vectors

In [44]:
vectorizer=CountVectorizer()
vectorizer.fit(X_train)

CountVectorizer()

count vector methodu ile x_train ve x_test verileri luşturuldu

In [45]:
x_train_count=vectorizer.transform(X_train)
x_test_count=vectorizer.transform(X_test)

In [25]:
vectorizer.get_feature_names()[:10]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['aaliyah',
 'abandon',
 'abandoned',
 'abc',
 'abel',
 'abhorrent',
 'abhors',
 'abiding',
 'ability',
 'abject']

In [26]:
x_train_count.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [27]:
x_test_count.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

## TF_İDF

### word level

In [46]:
tf_idf_word_vectorizer=TfidfVectorizer()

In [47]:
tf_idf_word_vectorizer.fit(X_train)

TfidfVectorizer()

In [48]:
x_train_tf_idf_word=tf_idf_word_vectorizer.transform(X_train)
x_test_tf_idf_word=tf_idf_word_vectorizer.transform(X_test)

In [49]:
tf_idf_word_vectorizer.get_feature_names()[:10]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['aaa',
 'aaliyah',
 'abagnale',
 'abandon',
 'abandoned',
 'abbass',
 'abbott',
 'abbreviated',
 'abc',
 'abdul']

In [ ]:
x_train_tf_idf_word.toarray()

### ngram level

In [50]:
tf_idf_ngram_vectorizer= TfidfVectorizer(ngram_range=(2,3))
tf_idf_ngram_vectorizer.fit(X_train)

TfidfVectorizer(ngram_range=(2, 3))

In [51]:
x_train_tf_idf_ngram=tf_idf_ngram_vectorizer.transform(X_train)
x_test_tf_idf_ngram=tf_idf_ngram_vectorizer.transform(X_test)

### charakter level

In [52]:
tf_idf_char_vectorizer=TfidfVectorizer(analyzer="char",ngram_range=(2,3))
tf_idf_char_vectorizer.fit(X_train)

TfidfVectorizer(analyzer='char', ngram_range=(2, 3))

In [53]:
x_train_tf_idf_char=tf_idf_char_vectorizer.transform(X_train)
x_test_tf_idf_char=tf_idf_char_vectorizer.transform(X_test)

#MAKİNE ÖĞRENMESİ İLE SENTİMENT SINIFLANDIRMASI

## LOJİSTİK REG.

### count vectors method

In [82]:
loj=linear_model.LogisticRegression()
loj_model=loj.fit(x_train_count,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [83]:
accuracy=model_selection.cross_val_score(loj_model,
                                         x_test_count,
                                         y_test,
                                         cv=10).mean()

In [56]:
print("count vectors dogruluk oranı:",accuracy)

count vectors dogruluk oranı: 0.8388075313807531


### tf-idf word method

In [57]:
loj=linear_model.LogisticRegression()
loj_model=loj.fit(x_train_tf_idf_word,y_train)
accuracy=model_selection.cross_val_score(loj_model,
                                         x_test_tf_idf_word,
                                         y_test,
                                         cv=10).mean()

print("tf_idf word level dogruluk oranı:",accuracy)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


tf_idf word level dogruluk oranı: 0.8353556485355649


### th-idf ngram method

In [58]:
loj=linear_model.LogisticRegression()
loj_model=loj.fit(x_train_tf_idf_ngram,y_train)
accuracy=model_selection.cross_val_score(loj_model,
                                         x_test_tf_idf_ngram,
                                         y_test,
                                         cv=10).mean()

print("tf_idf ngram level dogruluk oranı:",accuracy)

tf_idf ngram level dogruluk oranı: 0.7473849372384936


### tf-idf character method

In [59]:
loj=linear_model.LogisticRegression()
loj_model=loj.fit(x_train_tf_idf_char,y_train)
accuracy=model_selection.cross_val_score(loj_model,
                                         x_test_tf_idf_char,
                                         y_test,
                                         cv=10).mean()

print("tf_idf char level dogruluk oranı:",accuracy)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


tf_idf char level dogruluk oranı: 0.7846757322175733


## NAİVE BAYES

### count vectors  method

In [64]:
nb=naive_bayes.MultinomialNB()
nb_model=nb.fit(x_train_count,y_train)
accuracy=model_selection.cross_val_score(nb_model,
                                         x_test_count,
                                         y_test,
                                         cv=10).mean()

print("count vectors accuracy oranı: ",accuracy)

count vectors accuracy oranı:  0.8339435146443515


### tf-idf word method

In [65]:
nb=naive_bayes.MultinomialNB()
nb_model=nb.fit(x_train_tf_idf_word,y_train)
accuracy=model_selection.cross_val_score(nb_model,
                                         x_test_tf_idf_word,
                                         y_test,
                                         cv=10).mean()

print("naive bayes tf_idf word accuracy oranı: ",accuracy)



naive bayes tf_idf word accuracy oranı:  0.835826359832636


### tf-idf ngrams method

In [66]:
nb=naive_bayes.MultinomialNB()
nb_model=nb.fit(x_train_tf_idf_ngram,y_train)
accuracy=model_selection.cross_val_score(nb_model,
                                         x_test_tf_idf_ngram,
                                         y_test,
                                         cv=10).mean()

print("naive bayes tf_idf ngram accuracy oranı: ",accuracy)

naive bayes tf_idf ngram accuracy oranı:  0.7669456066945608


###tf-idf char method

In [67]:
nb=naive_bayes.MultinomialNB()
nb_model=nb.fit(x_train_tf_idf_char,y_train)
accuracy=model_selection.cross_val_score(nb_model,
                                         x_test_tf_idf_char,
                                         y_test,
                                         cv=10).mean()

print("naive bayes tf_idf char accuracy oranı: ",accuracy)

naive bayes tf_idf char accuracy oranı:  0.7568514644351465


## RANDOM FORESTS

### count method

In [ ]:
rf=ensemble.RandomForestClassifier()
rf_model=rf.fit(x_train_count,y_train)
accuracy=model_selection.cross_val_score(rf_model,
                                         x_test_count,
                                         y_test,
                                         cv=10).mean()

print("rf count method accuracy oranı:",accuracy)

###tf-idf word method

In [ ]:
rf=ensemble.RandomForestClassifier()
rf_model=rf.fit(x_train_tf_idf_word,y_train)
accuracy=model_selection.cross_val_score(rf_model,
                                         x_test_tf_idf_word,
                                         y_test,
                                         cv=10).mean()

print("rf tf-idf word method accuracy oranı:",accuracy)

###tf-idf ngram method

In [ ]:
rf=ensemble.RandomForestClassifier()
rf_model=rf.fit(x_train_tf_idf_ngram,y_train)
accuracy=model_selection.cross_val_score(rf_model,
                                         x_test_tf_idf_ngram,
                                         y_test,
                                         cv=10).mean()

print("rf tf-idf ngram method accuracy oranı:",accuracy)

###tf-idf char method

In [ ]:
rf=ensemble.RandomForestClassifier()
rf_model=rf.fit(x_train_tf_idf_char,y_train)
accuracy=model_selection.cross_val_score(rf_model,
                                         x_test_tf_idf_char,
                                         y_test,
                                         cv=10).mean()

print("rf tf-idf char method accuracy oranı:",accuracy)

## xgboost

###count method

In [78]:
xgb=xgboost.XGBClassifier()
xgb_model=xgb.fit(x_train_count,y_train)
accuracy=model_selection.cross_val_score(xgb_model,
                                         x_test_count,
                                         y_test,
                                         cv=5).mean()
print("xgboot count method accuracy oranı:",accuracy)

xgboot count method accuracy oranı: 0.6300732217573222


tf_idf word method

In [79]:
xgb=xgboost.XGBClassifier()
xgb_model=xgb.fit(x_train_tf_idf_word,y_train)
accuracy=model_selection.cross_val_score(xgb_model,
                                         x_test_tf_idf_word,
                                         y_test,
                                         cv=5).mean()
print("xgboot count method accuracy oranı:",accuracy)

xgboot count method accuracy oranı: 0.631276150627615


### tf_idf char method

In [80]:
xgb=xgboost.XGBClassifier()
xgb_model=xgb.fit(x_train_tf_idf_char,y_train)
accuracy=model_selection.cross_val_score(xgb_model,
                                         x_test_tf_idf_char,
                                         y_test,
                                         cv=5).mean()
print("xgboot tf_idf char method accuracy oranı:",accuracy)

xgboot tf_idf char method accuracy oranı: 0.6861924686192469


### tf_idf ngrams method

In [81]:
xgb=xgboost.XGBClassifier()
xgb_model=xgb.fit(x_train_tf_idf_ngram,y_train)
accuracy=model_selection.cross_val_score(xgb_model,
                                         x_test_tf_idf_ngram,
                                         y_test,
                                         cv=5).mean()
print("xgboot tf-idf ngrams method accuracy oranı:",accuracy)

xgboot tf-idf ngrams method accuracy oranı: 0.5636506276150628


## prediction 

yeni yorumu direkt modele veremeyiz modelin beslendii train verisi gibi yeni yorumlar da countvectorizer() dan geçmeli.
* tehmin işlemini logistic regresyonda uygulayacagız
*linear logistic reg. count methodu en iyi sonuc veren yöntem gibi duruyor 8388 en son o modeli tekrar çalıştırıp tahmine başlıyoruz


In [84]:
loj_model.predict("nice good i like it so much")
# hata çünkü vektörize edilmedi

ValueError: ignored

In [98]:
yeni_yorum="i lie it nice it is very fun"

In [165]:
yeni_yorum=pd.Series("i lie it nice it is very fun")
yorum2=pd.Series("this shit is awsome ")
yorum3=pd.Series("what a hilorius fuck")
yorum4=pd.Series("amazing film")
yorum5=pd.Series("no no no way i haven't see a film better than that")


In [166]:
vectorizer=CountVectorizer()

In [167]:
vectorizer.fit(X_train)

CountVectorizer()

In [168]:
yorum5=vectorizer.transform(yorum5)

In [169]:
loj_model.predict(yorum5)

array([1])